# Getting Started with TrdeStrats

This notebook demonstrates the complete workflow:
1. Fetch OHLCV data (BTC/USDT, 1h)
2. Apply the SMA Crossover strategy
3. Run a backtest
4. Visualize results

In [ ]:
from tradestrats.data.fetcher import fetch_ohlcv
from tradestrats.strategies.sma_cross import SMACrossover
from tradestrats.backtesting.engine import run
from tradestrats.visualization.charts import plot_candlestick, plot_equity_curve, plot_signals

## 1. Fetch Data

Fetch BTC/USDT 1h candles for the last 6 months from Binance.
Data is automatically cached as Parquet.

In [ ]:
from datetime import datetime, timedelta

end = datetime.utcnow()
start = end - timedelta(days=180)

data = fetch_ohlcv("BTC/USDT", timeframe="1h", start=start, end=end)
print(f"Fetched {len(data)} candles")
data.tail()

## 2. Apply Strategy

Use the SMA Crossover strategy with fast=20, slow=50 periods.

In [ ]:
strategy = SMACrossover(fast_period=20, slow_period=50)
print(strategy)

signals = strategy.generate_signals(data)
signals[["close", "sma_fast", "sma_slow", "signal"]].tail(10)

## 3. Run Backtest

In [ ]:
result = run(strategy, data, init_cash=10_000, fees=0.001)

for key, value in result.summary().items():
    print(f"{key}: {value:.4f}" if isinstance(value, float) else f"{key}: {value}")

## 4. Visualize Results

In [ ]:
# Candlestick chart with SMA overlays
fig = plot_candlestick(
    data,
    indicators={"SMA Fast": signals["sma_fast"], "SMA Slow": signals["sma_slow"]},
    title="BTC/USDT 1H with SMA Crossover",
)
fig.show()

In [ ]:
# Buy/Sell signals on price chart
fig = plot_signals(data, signals, title="SMA Crossover Signals")
fig.show()

In [ ]:
# Equity curve
fig = plot_equity_curve(result.equity_curve, title="Portfolio Equity Curve")
fig.show()